In [6]:
import pandas as pd
import numpy as np

In [7]:
#Load Dataset
test = pd.read_csv('/home/akshay/Documents/DBNKDD/dataset/NSL-KDD_Processed/Kdd_Test_41.csv')             # test set 
testLables = pd.read_csv('/home/akshay/Documents/DBNKDD/dataset/NSL-KDD_Processed/NSL_TestLabels_mat5.csv')     # test labels
train = pd.read_csv('/home/akshay/Documents/DBNKDD/dataset/NSL-KDD_Processed/Kdd_Train_41.csv')            # train set 
trainLables = pd.read_csv('/home/akshay/Documents/DBNKDD/dataset/NSL-KDD_Processed/NSL_TrainLabels_mat5.csv')    # train labels
valid = pd.read_csv('/home/akshay/Documents/DBNKDD/dataset/NSL-KDD_Processed/Kdd_Valid_41.csv')            # valid set
validLabels = pd.read_csv('/home/akshay/Documents/DBNKDD/dataset/NSL-KDD_Processed/NSL_ValidLabels_int3.csv')    # valid labels
trainLabelsOHE = pd.read_csv('/home/akshay/Documents/DBNKDD/dataset/NSL-KDD_Processed/NSL_TrainLabels_int.csv')
testLabelsOHE = pd.read_csv('/home/akshay/Documents/DBNKDD/dataset/NSL-KDD_Processed/NSL_TestLabels_int.csv')

test_set = np.float32(test)
test_labels_set = np.float32(testLables)
train_set = np.float32(train)
train_labels_set = np.float32(trainLables)
valid_set = np.float32(valid)
valid_labels_set = np.float32(validLabels)

In [8]:
import keras

Using CNTK backend
/home/akshay/.local/lib/python3.5/site-packages/keras/backend/cntk_backend.py:18: UserWarning: CNTK backend warning: GPU is not detected. CNTK's CPU version is not fully optimized,please run with GPU to get better performance.
  'CNTK backend warning: GPU is not detected. '


In [118]:
from keras.layers import Input, Dense
from keras.models import Model
from keras import optimizers

features = 41

input_layer = Input(shape=(41,))

encoded_layer_1 = Dense(features, activation='elu')(input_layer)
encoded_layer_2 = Dense(30, activation='elu')(encoded_layer_1)
encoded_layer_3 = Dense(20, activation='elu')(encoded_layer_2)
encoded_layer_4 = Dense(10, activation='elu')(encoded_layer_3)
encoded_layer_5 = Dense(5, activation='softmax')(encoded_layer_4)


decoded = Dense(5, activation='elu')(encoded_layer_5)
decoded = Dense(10, activation='elu')(encoded_layer_5)

decoded = Dense(20, activation='elu')(decoded)

decoded = Dense(30, activation='elu')(decoded)
decoded = Dense(41, activation='elu')(decoded)


# this model maps an input to its reconstruction
autoencoder = Model(input_layer, encoded_layer_5)
encoded = Model(input_layer, encoded_layer_5)
s = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=1, mode='auto')

/home/akshay/.local/lib/python3.5/site-packages/keras/backend/cntk_backend.py:2337: UserWarning: CNTK backend warning: CNTK version not detected. Will using CNTK 2.0 GA as default.
  'CNTK backend warning: CNTK version not detected. '


In [120]:
opt = keras.optimizers.Adam(lr=0.001,  decay=0.1)

In [130]:
autoencoder.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

/home/akshay/.local/lib/python3.5/site-packages/keras/backend/cntk_backend.py:2337: UserWarning: CNTK backend warning: CNTK version not detected. Will using CNTK 2.0 GA as default.
  'CNTK backend warning: CNTK version not detected. '


In [131]:

autoencoder.fit(train_set,train_labels_set,
                epochs=10,
                batch_size=256,
                shuffle=True,
                validation_split=0.1, callbacks=[s])

Train on 113374 samples, validate on 12598 samples
Epoch 1/10
113374/113374 [==============================] - 7s 61us/step - loss: 0.2628 - acc: 0.9353 - val_loss: 0.2585 - val_acc: 0.9362
Epoch 2/10
113374/113374 [==============================] - 16s 140us/step - loss: 0.2563 - acc: 0.9374 - val_loss: 0.2537 - val_acc: 0.9371
Epoch 3/10
113374/113374 [==============================] - 21s 185us/step - loss: 0.2523 - acc: 0.9382 - val_loss: 0.2502 - val_acc: 0.9379
Epoch 4/10
113374/113374 [==============================] - 12s 106us/step - loss: 0.2492 - acc: 0.9385 - val_loss: 0.2473 - val_acc: 0.9383
Epoch 5/10
113374/113374 [==============================] - 12s 103us/step - loss: 0.2466 - acc: 0.9393 - val_loss: 0.2448 - val_acc: 0.9388
Epoch 6/10
113374/113374 [==============================] - 14s 125us/step - loss: 0.2443 - acc: 0.9397 - val_loss: 0.2426 - val_acc: 0.9390
Epoch 7/10
113374/113374 [==============================] - 17s 150us/step - loss: 0.2423 - acc: 0.9400 -

In [133]:
_,a = autoencoder.evaluate(test_set, test_labels_set)

22542/22542 [==============================] - 3s 132us/step


In [135]:
ped = encoded.predict(test)


/home/akshay/.local/lib/python3.5/site-packages/cntk/core.py:361: UserWarning: your data is of type "float64", but your input variable (uid "Input10042") expects "<class 'numpy.float32'>". Please convert your data beforehand to speed up training.
  (sample.dtype, var.uid, str(var.dtype)))


In [137]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y, predict_op,target_names=['Normal', 'DoS','Probe','U2r', 'R2L']))

             precision    recall  f1-score   support

     Normal       0.66      0.95      0.77     10003
        DoS       0.87      0.74      0.80      7164
      Probe       0.88      0.74      0.80      2421
        U2r       0.00      0.00      0.00      2887
        R2L       0.00      0.00      0.00        67

avg / total       0.66      0.73      0.68     22542



/home/akshay/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [141]:
print(confusion_matrix(y, predict_op))

[[9463  357  183    0    0]
 [1865 5267   32    0    0]
 [ 194  445 1782    0    0]
 [2848    6   33    0    0]
 [  67    0    0    0    0]]
